In [1]:
import pandas as pd

# Washington State Patrol Traffic Stops, December 2008 to March 2016
Source: [The Stanford Open Policing Project](https://openpolicing.stanford.edu/data/)

In [3]:
df = pd.read_csv('data/input/wa_statewide_2019_02_25.csv')

## Clean data

Drop cases for which subject race is absent

In [4]:
df = df.dropna(subset=['subject_race'])

`type` and `department_name` are redundant; they are always '' and 'Washington State Patrol' respectively.

In [5]:
df = df.drop(['type', 'department_name', 'arrest_made', 'citation_issued', 'warning_issued'],axis=1)

In [6]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')

## Transform for sharing subsets

### Sample of raw data

In [7]:
df[df['date'] == '2016-03-31'].to_csv('data/output/sample_of_raw_data.csv', index=False)

### Statewide aggregated

In [9]:
df['year'] = df['date'].dt.year

In [10]:
df_aggregated = df.groupby([
    'year', 'county_name', 'subject_race', 'subject_sex', 'officer_race',
    'officer_sex'
])['raw_row_number'].count().reset_index()

In [11]:
df_aggregated.rename(columns={'raw_row_number': 'number_of_stops'}, inplace=True)

In [12]:
df_aggregated.to_csv('data/output/stops_aggregated.csv', index=False)

### King County

In [13]:
df_king_co = df[df['county_name']=='King']

In [14]:
df_king_co_subset = df_king_co.drop(['raw_row_number', 'contraband_found', 'subject_sex',
    'county_name', 'time', 'location', 'lat', 'lng', 'subject_age',
    'officer_race', 'officer_sex', 'search_conducted'
],
                             axis=1)

In [15]:
df_king_co_subset = df_king_co_subset[df_king_co_subset['date'] > '2015-03-31']

In [16]:
df_king_co_subset.to_csv('data/output/king_co.csv', index=False)